In [2]:
import numpy as np

# Load .npz file

archive_title_npz = np.load('movie_titles.npz')
archive_movie_npz= np.load('movie_recommend.npz')

# title keys
print("Claves disponibles en el archivo de títulos:", archive_title_npz.files)

# recommend keys
print("Claves disponibles en el archivo de recomendaciones de películas:", archive_movie_npz.files)


# Get title array
title_array = archive_title_npz['titles']
print("Forma del array de títulos:", title_array.shape)
print("Tipo de datos del array de títulos:", title_array.dtype)
print("Tamaño total del array de títulos:", title_array.size)

# Get recommendation array
movie_array = archive_movie_npz['data']
print("Forma del array de recomendaciones de películas:", movie_array.shape)
print("Tipo de datos del array de recomendaciones de películas:", movie_array.dtype)
print("Tamaño total del array de recomendaciones de películas:", movie_array.size)

# Get info
format_array = archive_movie_npz['format']
print("Formato del array de recomendaciones de películas:", format_array)

shape_array = archive_movie_npz['shape']
print("Forma del array de recomendaciones de películas:", shape_array)

row_array = archive_movie_npz['row']
print("Forma del array de índices de fila:", row_array.shape)
print("Tipo de datos del array de índices de fila:", row_array.dtype)
print("Tamaño total del array de índices de fila:", row_array.size)

col_array = archive_movie_npz['col']
print("Forma del array de índices de columna:", col_array.shape)
print("Tipo de datos del array de índices de columna:", col_array.dtype)
print("Tamaño total del array de índices de columna:", col_array.size)



Claves disponibles en el archivo de títulos: ['titles']
Claves disponibles en el archivo de recomendaciones de películas: ['format', 'shape', 'data', 'row', 'col']
Forma del array de títulos: (3706,)
Tipo de datos del array de títulos: <U75
Tamaño total del array de títulos: 3706
Forma del array de recomendaciones de películas: (1000209,)
Tipo de datos del array de recomendaciones de películas: int8
Tamaño total del array de recomendaciones de películas: 1000209
Formato del array de recomendaciones de películas: coo
Forma del array de recomendaciones de películas: [6040 3706]
Forma del array de índices de fila: (1000209,)
Tipo de datos del array de índices de fila: int32
Tamaño total del array de índices de fila: 1000209
Forma del array de índices de columna: (1000209,)
Tipo de datos del array de índices de columna: int32
Tamaño total del array de índices de columna: 1000209


In [16]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds

# Load data
archive_title_npz = np.load('movie_titles.npz')
archive_movie_npz = np.load('movie_recommend.npz')

# Recommend matrix
row = archive_movie_npz['row']
col = archive_movie_npz['col']
data = archive_movie_npz['data']
shape = archive_movie_npz['shape']
ratings_matrix = coo_matrix((data, (row, col)), shape=shape).astype(float)

# SVD
U, sigma, Vt = svds(ratings_matrix, k=10)

# Truncate matrix
sigma = np.diag(sigma)
ratings_truncated = np.dot(np.dot(U, sigma), Vt)

# Get title list
titles = archive_title_npz['titles']

def recomendar_peliculas(user_ratings, num_recomendaciones=100):

    # Movies dont viewed
    scores = np.dot(ratings_truncated[user_ratings], Vt.T)

    # Movies position by punctuation
    ranked_movies = np.argsort(-scores, axis=1)

    # Get best recommendation for each user
    recommendations = [titles[ranked_movies[i, :num_recomendaciones]] for i in range(len(user_ratings))]
    return recommendations


user_ratings = [5,5,5,5,5,5,1,1,1]
recomendaciones = recomendar_peliculas(user_ratings)
print(recomendaciones)



[array(['Wizard of Oz, The (1939)', 'Ben-Hur (1959)',
       'Christmas Story, A (1983)', 'James and the Giant Peach (1996)',
       "One Flew Over the Cuckoo's Nest (1975)",
       'Snow White and the Seven Dwarfs (1937)', 'My Fair Lady (1964)',
       'Erin Brockovich (2000)', 'Princess Bride, The (1987)',
       "Bug's Life, A (1998)"], dtype='<U75'), array(['Wizard of Oz, The (1939)', 'Ben-Hur (1959)',
       'Christmas Story, A (1983)', 'James and the Giant Peach (1996)',
       "One Flew Over the Cuckoo's Nest (1975)",
       'Snow White and the Seven Dwarfs (1937)', 'My Fair Lady (1964)',
       'Erin Brockovich (2000)', 'Princess Bride, The (1987)',
       "Bug's Life, A (1998)"], dtype='<U75'), array(['Wizard of Oz, The (1939)', 'Ben-Hur (1959)',
       'Christmas Story, A (1983)', 'James and the Giant Peach (1996)',
       "One Flew Over the Cuckoo's Nest (1975)",
       'Snow White and the Seven Dwarfs (1937)', 'My Fair Lady (1964)',
       'Erin Brockovich (2000)', 'Princes